![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**0- Connect to your Google Drive folder**

In [ ]:
my_folder = "/content/drive/MyDrive/COLAB-SAS"

from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/COLAB-SAS


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 5.2 MB/s 
  Created wheel for saspy: filename=saspy-4.4.1-py3-none-any.whl size=9937304 sha256=f6e8f6d40182ba70a476d8f838df970ad6027ab2a90ae8748b2519c7be7fc8da
  Stored in directory: /root/.cache/pip/wheels/0a/f6/ab/f7173c8a824856c66f6fd50194bb8fd3ef93fb4f1824be8d0c
Successfully built saspy


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
#'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

If there is no .authinfo file, you can create this

In [ ]:
#%%writefile .authinfo
#oda user USR password PSW

Copy this file to home

In [ ]:
!cp .authinfo ~/.authinfo

**5- Establish Connection (Need to do this step each time you use SASPy)**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 395



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/COLAB-SAS/sascfg_personal.py
WORK Path             = /saswork/SAS_work7227000163AC_odaws02-usw2.oda.sas.com/SAS_work4193000163AC_odaws02-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 91052


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [ ]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 04:39:00 AM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 04:39:00 AM

33         


For local Jupyter

In [ ]:
#%%SAS sas_session
#libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [ ]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
34823,2034824.0,ROMERO,ASHLEY,1.0,17.0,1000.0,University / College,City,7.0,1.0,...,2.0,2.0,30.0,10.0,20.0,0.0,0.0,1.0,1.0,20.0
341990,2341991.0,LYNCH,ARTHUR,0.0,22.0,114300.0,University / College,City,10.0,0.0,...,1.0,4.0,20.0,20.0,20.0,0.0,0.0,0.0,1.0,90.0


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [ ]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)  

data_mergefloats = data_merge.loc[:, ~data_merge.columns.isin(['LastName', 'FirstName', 'Education', 'City'])]

In [ ]:
data_mergefloats.isna().sum()

ID                  0
Woman               0
Age                 0
Salary              0
SeniorList          0
NbActivities        0
Referrals           0
Recency         81024
Frequency       81024
Seniority       81024
TotalGift       81024
MinGift             0
MaxGift         81024
GaveLastYear        0
AmtLastYear         0
Contact             0
GaveThisYear        0
AmtThisYear         0
dtype: int64

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(imputation_order='ascending',max_iter=15,random_state=13,n_nearest_features=10)

In [ ]:
for i in data_mergefloats:
  data_mergefloats.loc[:,i]=imputer.fit_transform(data_mergefloats[[i]])

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [ ]:
data_mergefloats = data_mergefloats.sort_index()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder_label = LabelEncoder()
data_mergefloats['City'] = encoder_label.fit_transform(data_merge['City'])
data_mergefloats['Education'] = encoder_label.fit_transform(data_merge['Education'])

In [ ]:
abs(data_mergefloats.corr()['AmtThisYear'])

ID              0.000189
Woman           0.043386
Age             0.038916
Salary          0.067602
SeniorList      0.004176
NbActivities    0.016713
Referrals       0.014993
Recency         0.005396
Frequency       0.009791
Seniority       0.006031
TotalGift       0.029114
MinGift         0.013452
MaxGift         0.027728
GaveLastYear    0.009698
AmtLastYear     0.012021
Contact         0.000929
GaveThisYear         NaN
AmtThisYear     1.000000
City            0.008818
Education       0.023672
Name: AmtThisYear, dtype: float64

## Data Partition

In [ ]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_mergefloats, test_size=0.1,random_state=13) # you can change the percentage
train.sample(5)

,ID,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear,City,Education
839753,2839754.0,1.0,67.0,10000.0,5.0,2.0,2.0,3.00000,2.000000,4.000000,45.000000,20.0,25.000000,0.0,0.0,0.0,1.0,10.0,3,1
531813,2531814.0,1.0,43.0,152900.0,4.0,0.0,0.0,2.75091,1.985796,4.674952,134.864685,0.0,106.674485,0.0,0.0,0.0,1.0,30.0,1,2
962146,2962147.0,0.0,31.0,110600.0,0.0,0.0,0.0,2.75091,1.985796,4.674952,134.864685,0.0,106.674485,0.0,0.0,1.0,1.0,20.0,2,2
926887,2926888.0,1.0,66.0,23800.0,10.0,0.0,0.0,9.00000,1.000000,9.000000,20.000000,20.0,20.000000,0.0,0.0,1.0,1.0,20.0,1,0
717325,2717326.0,1.0,90.0,1000.0,6.0,0.0,0.0,2.75091,1.985796,4.674952,134.864685,0.0,106.674485,0.0,0.0,0.0,1.0,10.0,3,1



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [ ]:
from sklearn import linear_model

#comment: it's numpy array
#X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities', 'Education','TotalGift','City' ]]
X_train = train[['Salary','TotalGift']] 
Y_train = train['AmtThisYear']
#X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities','Education','TotalGift','City']] 
X_valid = validation[['Salary','TotalGift']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[53.55408572 70.5715051  66.88926677 ... 72.76027854 78.26605382
 62.28650139]


In [ ]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

61.887125825360535
46214.75483778357
214.97617272103338


## Regression Tree Model（Py）

In [ ]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities' ]] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=5, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[42.59022696 57.67645276 61.35533846 ... 61.35533846 65.95705087
 93.89675195]


In [ ]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

62.04433015237193
45634.612886309755
213.6225945126352


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


## Scoring New Data

### Prepare data for scoring

In [ ]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [ ]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

In [ ]:
scoring_data_contact[['MinGift']] = scoring_data_contact[['MinGift']].fillna(value=0) 
scoring_datafloats = scoring_data_contact.loc[:, ~scoring_data_contact.columns.isin(['LastName', 'FirstName', 'Education', 'City'])]

In [ ]:
scoring_datafloats

,ID,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact
0,2000001.0,0.0,25.0,107200.0,2.0,0.0,0.0,1.0,2.0,2.0,1010.0,10.0,1000.0,0.0,0.0,1.0
1,2000002.0,1.0,38.0,15800.0,4.0,1.0,1.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,1.0
2,2000003.0,1.0,37.0,57400.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,1.0
3,2000004.0,1.0,78.0,23700.0,3.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,1.0
4,2000005.0,1.0,34.0,71900.0,8.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2999996.0,1.0,29.0,15100.0,8.0,3.0,2.0,5.0,1.0,5.0,20.0,20.0,20.0,0.0,0.0,1.0
999996,2999997.0,1.0,22.0,7000.0,10.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,1.0
999997,2999998.0,0.0,17.0,1000.0,10.0,1.0,1.0,1.0,1.0,1.0,20.0,20.0,20.0,0.0,0.0,1.0
999998,2999999.0,1.0,32.0,43900.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,1.0


In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imputer = IterativeImputer(imputation_order='ascending',max_iter=15,random_state=13,n_nearest_features=10)

In [ ]:
for i in scoring_datafloats:
  scoring_datafloats.loc[:,i]=imputer.fit_transform(scoring_datafloats[[i]])

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [ ]:
scoring_datafloats = scoring_datafloats.sort_index()

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder_label = LabelEncoder()
scoring_datafloats['City'] = encoder_label.fit_transform(scoring_data_contact['City'])
scoring_datafloats['Education'] = encoder_label.fit_transform(scoring_data_contact['Education'])

In [ ]:
scoring_datafloats

,ID,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,City,Education
0,2000001.0,0.0,25.0,107200.0,2.0,0.0,0.0,1.000000,2.000000,2.000000,1010.000000,10.0,1000.000000,0.0,0.0,1.0,0,2
1,2000002.0,1.0,38.0,15800.0,4.0,1.0,1.0,2.165225,1.717504,3.785557,107.680667,0.0,87.482711,0.0,0.0,1.0,2,1
2,2000003.0,1.0,37.0,57400.0,5.0,0.0,0.0,2.165225,1.717504,3.785557,107.680667,0.0,87.482711,0.0,0.0,1.0,2,2
3,2000004.0,1.0,78.0,23700.0,3.0,0.0,0.0,2.165225,1.717504,3.785557,107.680667,0.0,87.482711,0.0,0.0,1.0,2,1
4,2000005.0,1.0,34.0,71900.0,8.0,0.0,0.0,2.165225,1.717504,3.785557,107.680667,0.0,87.482711,0.0,0.0,1.0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2999996.0,1.0,29.0,15100.0,8.0,3.0,2.0,5.000000,1.000000,5.000000,20.000000,20.0,20.000000,0.0,0.0,1.0,3,1
999996,2999997.0,1.0,22.0,7000.0,10.0,0.0,0.0,2.165225,1.717504,3.785557,107.680667,0.0,87.482711,0.0,0.0,1.0,3,1
999997,2999998.0,0.0,17.0,1000.0,10.0,1.0,1.0,1.000000,1.000000,1.000000,20.000000,20.0,20.000000,0.0,0.0,1.0,0,1
999998,2999999.0,1.0,32.0,43900.0,0.0,0.0,0.0,2.165225,1.717504,3.785557,107.680667,0.0,87.482711,0.0,0.0,1.0,2,2


In [ ]:
#scoring_data_contact.head()

X = scoring_data_contact[['Salary','TotalGift']] 

regr_predict_contact=regr.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

ValueError: ignored

### Predict 'amount given' for members who were not contacted

In [ ]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['MinGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)

#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age', 'Salary','Contact','MinGift','AmtLastYear', 'GaveLastYear','Woman', 'NbActivities' ]] 

regr_predict_nocontact=regr.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtNoContact
0,2000001.0,70.772954
1,2000002.0,63.509692
2,2000003.0,72.994229
3,2000004.0,42.762552
4,2000005.0,78.502850


In [ ]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,68.007003,70.772954
1,2000002.0,60.743741,63.509692
2,2000003.0,70.228278,72.994229


## Exporting Results to a CSV File

In [ ]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.

In [ ]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,65.41565172857224,70.36563134075104
2000002.0,58.02239763644701,62.972377248625804
2000003.0,67.35718778191847,72.30716739409728
2000004.0,37.733731742296456,42.683711354475264
2000005.0,72.67422014920815,77.62419976138696
2000006.0,31.61555694917946,36.56553656135826
2000007.0,37.76026216815852,42.71024178033733
2000008.0,58.881840764009134,63.831820376187935
2000009.0,37.92037153344358,42.87035114562238
